# Lab in Data Science: Final Project

Pierre Fouche, Matthias Leroy and Raphaël Steinmann

## Imports

In [1]:
%matplotlib inline
import matplotlib.pylab as plt
import numpy as np
plt.rcParams['figure.figsize'] = (10,6)
plt.rcParams['font.size'] = 18
plt.style.use('fivethirtyeight')

In [39]:
import getpass
import pyspark
from datetime import datetime,timedelta
from pyspark.sql import SparkSession
import pyspark.sql.functions as functions
from pyspark.sql.types import BooleanType
from pyspark.sql.window import Window
import math
import helpers, spark_helpers
import pickle
import random

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Initialize the `SparkSession`

In [3]:
conf = pyspark.conf.SparkConf()
conf.setMaster('yarn')
conf.setAppName('project-{0}'.format(getpass.getuser()))
conf.set('spark.executor.memory', '6g')
conf.set('spark.executor.instances', '6')
conf.set('spark.port.maxRetries', '100')
sc = pyspark.SparkContext.getOrCreate(conf)
conf = sc.getConf()
sc

<SparkContext master=yarn appName=project-rbsteinm>

In [4]:
# init spark session
spark = SparkSession(sc)

## Data Processing

### Cleaning metadata
First, let's clean the metadata dataframe. We will use the SBB data limited around the Zurich area. We will focus on all the stops within 10km of the Zurich train station. Let's get rid of all the stations that are too far away from Zurich:

In [5]:
# load metadata
raw_metadata = spark.read.load('/datasets/project/metadata', format='com.databricks.spark.csv', header='false', sep='\\t')

# remove multiple spaces
metadata = raw_metadata.withColumn('_c0', functions.regexp_replace(raw_metadata._c0, '\s+', ' '))
# split into columns
metadata = metadata.withColumn('name', functions.split(metadata._c0, '%')[1])
for (name, index, type_) in [('station_ID',0, 'int'), ('long',1, 'double'), ('lat',2, 'double'), ('height',3, 'int')]:
    metadata = metadata.withColumn(name, functions.split(metadata._c0, ' ')[index].cast(type_))
# remove useless column
metadata = metadata.drop('_c0')
# trim name column to remove left/right blank
metadata = metadata.withColumn('name', functions.trim(metadata.name))

# coordinates of Zürich main train station
lat_zurich = 47.3782
long_zurich = 8.5402

# convert to pandas dataframe
pandas_df = metadata.toPandas()

# keep only the stops that are located < 10km from Zurich HB
pandas_df['distance_to_zh'] = pandas_df.apply(lambda x: helpers.distance(x['long'], x['lat'], long_zurich, lat_zurich), axis=1)
pandas_df = pandas_df[pandas_df['distance_to_zh'] < 10]

# recreate spark dataframe from pandas dataframe
metadata = spark.createDataFrame(pandas_df)
# create dict of stations from pandas dataframe
stations = pandas_df.set_index('station_ID').to_dict('index')

# dump metadata in pickle
with open('./data/metadata.pickle', 'wb') as handle:
    pickle.dump(stations, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [6]:
# load metadata from pickle
with open('./data/metadata.pickle', 'rb') as handle:
    stations = pickle.load(handle)

### Cleaning main dataset

In [7]:
# load full data
# raw_df = spark.read.load('/datasets/project/istdaten/*/*', format='csv', header='true', inferSchema='true', sep=';')
# load sample data
raw_df = spark.read.load('/datasets/project/istdaten/2018/01', format='csv', header='true', inferSchema='true', sep=';')

In [8]:
# rename the fields german -> english
fields = {
    'BETRIEBSTAG':'date',
    'FAHRT_BEZEICHNER':'trip_id',
    'PRODUKT_ID':'transport_type',
    'LINIEN_ID':'train_id',
    'LINIEN_TEXT':'line',
    'VERKEHRSMITTEL_TEXT':'train_type',
    'ZUSATZFAHRT_TF':'additional_trip',
    'FAELLT_AUS_TF':'trip_failed',
    'HALTESTELLEN_NAME':'stop_name',
    'BPUIC':'stop_id',
    'ANKUNFTSZEIT':'schedule_arrival',
    'AN_PROGNOSE':'real_arrival',
    'AN_PROGNOSE_STATUS':'arr_forecast_status',
    'ABFAHRTSZEIT':'schedule_dep',
    'AB_PROGNOSE':'real_dep',
    'AB_PROGNOSE_STATUS':'dep_forecast_status',
    'DURCHFAHRT_TF':'no_stop_here'
}

df = raw_df.selectExpr([k + ' as ' + fields[k] for k in fields])

In [9]:
# refactor dates
df = df.withColumn('date', functions.from_unixtime(functions.unix_timestamp('date', 'dd.MM.yyyy')))
df = df.withColumn('schedule_arrival', functions.from_unixtime(functions.unix_timestamp('schedule_arrival', 'dd.MM.yyyy HH:mm')))
df = df.withColumn('real_arrival', functions.from_unixtime(functions.unix_timestamp('real_arrival', 'dd.MM.yyyy HH:mm')))
df = df.withColumn('schedule_dep', functions.from_unixtime(functions.unix_timestamp('schedule_dep', 'dd.MM.yyyy HH:mm')))
df = df.withColumn('real_dep', functions.from_unixtime(functions.unix_timestamp('real_dep', 'dd.MM.yyyy HH:mm')))

In [10]:
# add a column containing the weekday (monday=1, sunday=6)
df = df.withColumn('weekday', spark_helpers.get_weekday(df.date))

In [11]:
# keep only the rows with stops near zurich
df = df.where(df.stop_id.isin([int(x) for x in list(pandas_df.station_ID.unique())]))

In [12]:
# there is still 51'571'541 rows in zurich area
# df.count()

In [13]:
# keep only date after the 10th of december, because the schedule changed
df = df.where(df.date > '2017-12-10 00:00:00')

In [14]:
# discard the rows when there is no stop here
df2 = df.where(df.no_stop_here == 'false')

In [15]:
# discard ill-formated rows where the train leaves a station before arriving in it
df2 = df2.where((df2.schedule_dep >= df2.schedule_arrival) | functions.col('schedule_arrival').isNull() | functions.col('schedule_dep').isNull())

In [97]:
df.count()

7207472

In [98]:
df2.count()

7207450

## Modeling the network

### From stops to trips

In [16]:
# create a column with the schedule time that will be used to build the network
df2 = df2.withColumn('schedule_time', spark_helpers.date_choice(df2.schedule_arrival, df2.schedule_dep))
#df2 = df2.withColumn('schedule_time', functions.from_unixtime(functions.unix_timestamp('schedule_time', 'dd.MM.yyyy HH:mm')))

# create a column that tells if a stop is the first/last one of its trip or in the middle
df2 = df2.withColumn('stop_type', spark_helpers.stop_type(df2.schedule_dep, df2.schedule_arrival))

In [17]:
trips = df2.select(['trip_id', 'date', 'schedule_time', 'stop_id', 'stop_type', 'schedule_arrival', 'schedule_dep', 'line']).orderBy(['trip_id', 'schedule_time'], ascending=[0,0])

In [18]:
# duplicate the dataframe, shift the copy of one row and append it to the original
# this way, we have for each row the current stop and the next stop
w = Window().partitionBy(functions.col('trip_id')).orderBy(functions.col('trip_id'))
trips2 = trips.select("*", functions.lag("trip_id").over(w).alias("next_tid"))
trips2 = trips2.select("*", functions.lag("schedule_time").over(w).alias("next_time"))
trips2 = trips2.select("*", functions.lag("stop_id").over(w).alias("next_sid"))
trips2 = trips2.select("*", functions.lag("stop_type").over(w).alias("next_type"))
trips2 = trips2.select("*", functions.lag("schedule_arrival").over(w).alias("next_sched_arr"))
trips2 = trips2.select("*", functions.lag("schedule_dep").over(w).alias("next_sched_dep"))

trips2 = trips2.where(trips2.next_time.isNotNull())

In [19]:
# create a new column telling if the edge is valid or not
# (i.e. if the stop and next stop are really part of the same ride)
trips3 = trips2.withColumn('is_valid', spark_helpers.edge_is_valid(trips2.trip_id, trips2.schedule_time, trips2.stop_id, trips2.stop_type, trips2.next_tid, trips2.next_time, trips2.next_sid, trips2.next_type, trips2.schedule_dep,trips2.next_sched_arr))

In [20]:
# keep only valid edges
trips4 = trips3.filter(trips3.is_valid=='true')

### For each day of the week, model the network
Get the edges of the network and the departure/arrival times for each trip (edge=trip)
We assume the schedule repeat every week, and we generate one schedule per weekday.
Days off have the same schedules as sundays.

In [21]:
# creating a model for each day of the week
# this code needs to be run only once
typical_monday = '2018-01-15 00:00:00'
typical_tuesday = '2018-01-16 00:00:00'
typical_wednesday = '2018-01-17 00:00:00'
typical_thursday = '2018-01-18 00:00:00'
typical_friday = '2018-01-19 00:00:00'
typical_saturday = '2018-01-20 00:00:00'
typical_sunday = '2018-01-21 00:00:00'
typical_week = [typical_monday,typical_tuesday,typical_wednesday,typical_thursday,typical_friday,typical_saturday,typical_sunday]

In [22]:
regenerate_models = False
days_names = ['monday','tuesday','wednesday','thursday','friday','saturday','sunday']

# generate one network for each weekday and store them in pickles
if regenerate_models:
    for (date, day_name) in zip(typical_week, days_names):
        network = (helpers.model_network(trips4, date))
        with open('./data/'+day_name+'.pickle', 'wb') as handle:
            helpers.network_to_datetime(network) # works inplace
            pickle.dump(network, handle, protocol=pickle.HIGHEST_PROTOCOL)
        print(str(day_name) + ' done')

In [23]:
# load the networks from the pickles
models = []
for day in days_names:
    with open('./data/'+ day +'.pickle', 'rb') as handle:
        network = pickle.load(handle)
    models.append(network)
    print(day + ' loaded')

monday loaded
tuesday loaded
wednesday loaded
thursday loaded
friday loaded
saturday loaded
sunday loaded


### Compute walking network

In [172]:
# compute walking network
walking_network = helpers.compute_walking_network(stations)
print('walking network loaded')

walking network loaded


## Shortest path algorithm

In [92]:
# example
sp = helpers.shortest_path(models, walking_network, stations,8576218, 8590727, datetime(2018, 1, 15, 14))
helpers.reduced_path_tostring(helpers.reduce_path(sp), stations)

line 916 from Küsnacht ZH, Bergstrasse to Zürich Tiefenbrunnen, Bahnhof 14:03 -> 14:16(13 stops)
line 4 from Zürich Tiefenbrunnen, Bahnhof to Zürich, Opernhaus 14:17 -> 14:24(6 stops)
line walk from Zürich, Opernhaus to Zürich Stadelhofen 14:24 -> 14:25(1 stops)
line S6 from Zürich Stadelhofen to Zürich Hardbrücke 14:27 -> 14:33(2 stops)
line S5 from Zürich Hardbrücke to Zürich Altstetten 14:41 -> 14:44(1 stops)
line walk from Zürich Altstetten to Zürich Altstetten, Bahnhof 14:44 -> 14:44(1 stops)
line 89 from Zürich Altstetten, Bahnhof to Zürich, Frankental 14:45 -> 14:51(5 stops)
line walk from Zürich, Frankental to Oberengstringen, Eggbühl 14:51 -> 14:57(1 stops)
line 308 from Oberengstringen, Eggbühl to Oberengstringen, Zentrum 15:02 -> 15:04(2 stops)


## Some tests

In [177]:
# select two stops at random and create a shortest path. Do that 100 times
shortest_paths = []
n_runs = 500
date = datetime(2018, 1, 16, 14)
source = 8503000 # zurich HB
reachable_stations_ids = helpers.get_reachable_stations(models[date.weekday()], walking_network, source)
reachable_stations = {sid: stations[sid] for sid in reachable_stations_ids}
for i in range(n_runs):
    if i%10==0:
        print(100*i/n_runs, '% finished')
    #source = random.choice(list(stations.keys()))
    dest = random.choice(list(reachable_stations.keys()))
    shortest_paths.append(helpers.shortest_path(models, walking_network, reachable_stations, source, dest, date))

0.0 % finished
2.0 % finished
4.0 % finished
6.0 % finished
8.0 % finished
10.0 % finished
12.0 % finished
14.0 % finished
16.0 % finished
18.0 % finished
20.0 % finished
22.0 % finished
24.0 % finished
26.0 % finished
28.0 % finished
30.0 % finished
32.0 % finished
34.0 % finished
36.0 % finished
38.0 % finished
40.0 % finished
42.0 % finished
44.0 % finished
46.0 % finished
48.0 % finished
50.0 % finished
52.0 % finished
54.0 % finished
56.0 % finished
58.0 % finished
60.0 % finished
62.0 % finished
64.0 % finished
66.0 % finished
68.0 % finished
70.0 % finished
72.0 % finished
74.0 % finished
76.0 % finished
78.0 % finished
80.0 % finished
82.0 % finished
84.0 % finished
86.0 % finished
88.0 % finished
90.0 % finished
92.0 % finished
94.0 % finished
96.0 % finished
98.0 % finished


## Shortest path with arrival time

In [209]:
def get_next_correspondance_reverse(edges, current_time, walking_network, source, dest, prev_tid):
    """
    returns departure/arrival times of the first ride departing after the current time
    assumes that the list current_time is sorted according to departure times!
    prev_tid: trip_id of the edge that led to source. Allows us to check if there is a change of bus at dest,
    in which case we will add one minute to the current time to take changing time into account
    
    Here since we are in reverse:
    - we follow the reverse edge from u to v
    - original edges goes v -> u, or dest -> source
    - source==u and dest==v
    - dep is the time at which you leave v and arr is the time at which you arrive at u
    """
    tid, line = None,None
    # if the edge exists for the rides (here source is v and dest is u)
    if source in edges and dest in edges[source]:
        times = edges[source][dest] # list of schedules form source to destinations
        # index of the fist ride arriving before the current time
        index = np.searchsorted([x[1] for x in times], current_time) - 1
        # None if there is no ride that can make you arrive at this time
        (dep,arr,tid,line) = times[index] if index >= 0 else (None,None,None,None)
        # If you have less than one minute for a change of vehicule, that's not acceptable
        # add 60 second to the current time and seach again for the next correspondance
        if tid is not None and prev_tid!=tid and current_time==arr:
            index = np.searchsorted([x[1] for x in times], current_time-timedelta(seconds=60)) - 1
            (dep,arr,tid,line) = times[index] if index >= 0 else (None,None,None,None)
    else:
        (dep,arr) = (None,None) # None if there is no edge by vehicule
        
    # determine if you can reach dest from source by foot
    if source in walking_network and dest in walking_network[source]:
        walk_time = walking_network[source][dest]
        (dep_walk,arr_walk) = (current_time-walk_time,current_time)
    else:
        (dep_walk,arr_walk) = (None,None)
    
    if dep is None and dep_walk is None:
        return (None,None,None,None) # if you can neither walk nor take a transport
    elif dep is None:
        return (dep_walk,arr_walk,'walk','walk') # if you can only walk
    elif dep_walk is None:
        return (dep,arr,tid,line) # if you can only take a transport
    else:
        return (dep,arr,tid,line) if (dep>=dep_walk) else (dep_walk,arr_walk,'walk','walk') # if you can walk or ride, do the fastest
    
def shortest_path_reverse(models, walking_network, stations, source, destination, arrival_time):
    '''
    Compute the shortest path between destination and source using Dijksta's algorithm
    models: contains 7 networks, one for each day of the week. They can be generated using model_network()
    source, destination: station IDs
    '''
    edges = build_reverse_network(models[arrival_time.weekday()]) # get the network for the correct day of the week
    Q = set(stations.keys()) # deep copy
    dist = dict.fromkeys(Q, datetime.min) # distances to the source (= departure time at each node)
    prev = dict.fromkeys(Q, (None, None, None, None, None)) # (previous node, dep/arr times, trip_id, line) in the shortest path
    dist[destination] = arrival_time
    
    while Q:
        unvisited_dist = {key: dist[key] for key in Q} # distances of unvisited nodes
        u = max(unvisited_dist, key=unvisited_dist.get) # u <- vertex in Q with maximum dist[u]
        #print('current node ',u)
        
        if dist[u] == datetime.min:
            raise Exception('Only nodes with infinity distance in the queue. The graph is disconected')
        
        Q.remove(u) #remove u from Q
        
        # if this is the source node, we can terminate
        if u == source:
            path = []
            # reconstruct the shortest path
            while prev[u][0] != destination:
                assert(prev[u][0] is not None),'no path from ' + stations[source].name + ' to ' + stations[destination].name
                assert(len(path) < 300), 'Path has more than 300 hops, something is wrong ...'
                current_edge = (prev[u][0],u,prev[u][1],prev[u][2],prev[u][3],prev[u][4])
                #path.insert(0,current_edge)
                path.append(current_edge)
                u = prev[u][0] # get previous node
            current_edge = (prev[u][0],u,prev[u][1],prev[u][2],prev[u][3],prev[u][4])
            #path.insert(0,current_edge) # push the source at the beginning of the path
            path.append(current_edge)
            return path
        
        current_time = dist[u]
        neighbors = set(edges[u].keys()) if u in edges else set() # u's neighbors by vehicule
        walk_neighbors = set(walking_network[u].keys())
        for v in neighbors.union(walk_neighbors):
            # take the first correspondance. dep is the departure time from v and arr the arrival time at u
            (dep_time,arr_time,tid,line) = get_next_correspondance_reverse(edges, current_time, walking_network, u, v, prev[u][3])
            # there is no more correspondance for this edge
            if dep_time is None:
                continue
            dist_u_v = arr_time - dep_time # travelling time from v to u
            waiting_time = current_time -  arr_time # waiting time after your arrival at u
            dep_v = current_time - waiting_time - dist_u_v # determine at what time you left v for u
            # a shorter path to v has been found
            if dep_v > dist[v]:
                dist[v] = dep_v
                prev[v] = (u,dep_time,arr_time,tid,line)
            
    raise Exception('No path was found from source to destination')

In [210]:
sp = shortest_path_reverse(models, walking_network, stations,8576218, 8590727, datetime(2018, 1, 15, 14))

In [211]:
sp

[(8576217,
  8576218,
  datetime.datetime(2018, 1, 15, 12, 29, 2, 47025),
  datetime.datetime(2018, 1, 15, 12, 34),
  'walk',
  'walk'),
 (8576216,
  8576217,
  datetime.datetime(2018, 1, 15, 12, 34),
  datetime.datetime(2018, 1, 15, 12, 35),
  '85:849:104820-01912-1',
  '916'),
 (8576209,
  8576216,
  datetime.datetime(2018, 1, 15, 12, 36, 55, 139052),
  datetime.datetime(2018, 1, 15, 12, 41, 57, 350685),
  'walk',
  'walk'),
 (8576208,
  8576209,
  datetime.datetime(2018, 1, 15, 12, 41, 57, 350685),
  datetime.datetime(2018, 1, 15, 12, 46, 20, 895690),
  'walk',
  'walk'),
 (8576207,
  8576208,
  datetime.datetime(2018, 1, 15, 12, 46, 20, 895690),
  datetime.datetime(2018, 1, 15, 12, 50, 34, 734539),
  'walk',
  'walk'),
 (8576206,
  8576207,
  datetime.datetime(2018, 1, 15, 12, 50, 34, 734539),
  datetime.datetime(2018, 1, 15, 12, 54),
  'walk',
  'walk'),
 (8576205,
  8576206,
  datetime.datetime(2018, 1, 15, 12, 54),
  datetime.datetime(2018, 1, 15, 12, 55),
  '85:849:104749-01912

In [184]:
def build_reverse_network(network):
    '''
    reverts all the edges in the network
    '''
    reverse_net = dict()
    for source in network.keys():
        for dest in network[source].keys():
            if not dest in reverse_net.keys():
                reverse_net[dest] = dict()
            reverse_net[dest][source] = network[source][dest]
    return reverse_net
reverse_net = build_reverse_network(net)

In [186]:
len(net)

870